In [ ]:
### Would-play baseline: just rank which games are popular and which are not, and return '1' if a game is among the top-ranked

gameCount = defaultdict(int)
totalPlayed = 0

for user,game,_ in readJSON("train.json.gz"):
  gameCount[game] += 1
  totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPlayed/2: break

predictions = open("predictions_Played.txt", 'w')
for l in open("pairs_Played.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,g = l.strip().split('-')
  if g in return1:
    predictions.write(u + '-' + g + ",1\n")
  else:
    predictions.write(u + '-' + g + ",0\n")

predictions.close()

### Category prediction baseline: Just consider some of the most common words from each category

catDict = {
  "Action": 0,
  "Strategy": 1,
  "RPG": 2,
  "Adventure": 3,
  "Sport": 4
}

predictions = open("predictions_Category.txt", 'w')
predictions.write("userID-reviewID,prediction\n")
for u,_,d in readJSON("test_Category.json.gz"):
  cat = catDict['Action'] # If there's no evidence, just choose the most common category in the dataset
  words = d['text'].lower()
  if 'strategy' in words:
    cat = catDict['Strategy']
  if 'rpg' in words:
    cat = catDict['RPG']
  if 'adventure' in words:
    cat = catDict['Adventure']
  if 'sport' in words:
    cat = catDict['Sport']
  predictions.write(u + '-' + d['reviewID'] + "," + str(cat) + "\n")

predictions.close()

In [6]:
import pandas as pd
import numpy as np
import gzip
from collections import defaultdict

In [23]:
def readJSON(path):
  for l in gzip.open(path, 'r+'):
    d = eval(l)
    u = d['userID']
    try:
      g = d['gameID']
    except Exception as e:
      g = None
    yield u,g,d
    
def parseData(path):
    for l in gzip.open(path, 'r+'):
        yield eval(l)

In [10]:
### Time-played baseline: compute averages for each user, or return the global average if we've never seen the user before
allHours = []
userHours = defaultdict(list)

for user,game,d in readJSON("train.json.gz"):
  h = d['hours_transformed']
  allHours.append(h)
  userHours[user].append(h)

In [11]:
globalAverage = sum(allHours) / len(allHours)
userAverage = {}
for u in userHours:
  userAverage[u] = sum(userHours[u]) / len(userHours[u])

predictions = open("predictions_Hours.txt", 'w')
for l in open("pairs_Hours.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,g = l.strip().split('-')
  if u in userAverage:
    predictions.write(u + '-' + g + ',' + str(userAverage[u]) + '\n')
  else:
    predictions.write(u + '-' + g + ',' + str(globalAverage) + '\n')

predictions.close()

In [30]:
parsedT = list(parseData('train.json.gz'))

In [31]:
train = []
val = []
for x in range(165000):
    train.append(parsedT[x])
for x in range(165000,175000):
    val.append(parsedT[x])

In [37]:
val

[{'hours': 5.7,
  'gameID': 'b25961467',
  'hours_transformed': 2.7441610955704103,
  'early_access': False,
  'date': '2017-11-23',
  'text': 'Kerbals need you, buy this game.  Enough said.',
  'userID': 'u49969792'},
 {'hours': 17.1,
  'text': 'This is the perfect gift for my friend!!!',
  'gameID': 'b73229067',
  'hours_transformed': 4.177917792195843,
  'early_access': False,
  'user_id': '76561198116282623',
  'date': '2017-12-24',
  'userID': 'u33147591'},
 {'hours': 9.9,
  'gameID': 'b37068085',
  'hours_transformed': 3.4462562298895643,
  'early_access': False,
  'date': '2017-09-30',
  'text': 'good game!',
  'userID': 'u00954406'},
 {'userID': 'u40416473',
  'early_access': False,
  'hours': 15.0,
  'hours_transformed': 4.0,
  'found_funny': 1,
  'text': 'Weird thing, but I like swords, so..yeah',
  'gameID': 'b85572219',
  'date': '2017-09-27'},
 {'hours': 348.3,
  'gameID': 'b88846011',
  'hours_transformed': 8.448322832507502,
  'early_access': False,
  'date': '2015-05-02

In [42]:
len(np.unique([d['userID'] for d in val]))

4766